## 数据集划分 21天预测未来7天的销售总量

-  test_data X[2016-09-27,2019-09-30;2016-10-08,2016-10-24] Y[2016-10-25,2016-10-31]

-  train1 X[2016-09-20，2016-09-30；2016-10-08，2016-10-17] Y[2016-10-18,2016-10-24]
- train2 X[2016-09-10,2016-09-30] Y[2016-10-08,2016-10-14]
- train3 X[2016-09-03,2016-09-23] Y[2016-09-24,2016-09-30]


In [1]:
import pandas as pd

In [2]:
test_train_start_date = '2016-09-27'
test_train_end_date = '2016-10-24'
test_test_start_date = '2016-10-25'
test_test_end_date = '2016-10-31'

In [3]:
# 全量时间窗口
all_date_range = [x.strftime('%Y-%m-%d') for x in pd.date_range(test_train_start_date,test_test_end_date)]
# 忽略十一假期
filter_date_range = [x.strftime('%Y-%m-%d') for x in pd.date_range('2016-10-01','2016-10-07')]
all_date_range = list(filter(lambda x: x not in filter_date_range,all_date_range))

# 训练集时间窗口
train_date_range = all_date_range[:21]
# 测试集时间窗口
test_date_range = all_date_range[21:]
print(all_date_range)
print(train_date_range)
print(test_date_range)

df_date = pd.DataFrame(all_date_range,columns = ['date'])
df_date['key']=0
assert df_date.shape[0]==28
df_date.head()

In [4]:
df_shop = pd.DataFrame(range(1, 2001), columns=['shop_id'])
df_shop['key'] = 0
df_dim = pd.merge(df_shop, df_date, on='key')
df_dim = df_dim[['shop_id', 'date']]
assert df_dim.shape[0] == 2000*28
df_dim.head()

In [5]:
df_pay = pd.read_csv('../data/pay_stat_day.csv')
df_pay.head()

,shop_id,date,pay_count
0,1,2015-10-10,188
1,1,2015-10-11,272
2,1,2015-10-12,257
3,1,2015-10-13,239
4,1,2015-10-14,277


In [6]:
df = pd.merge(df_dim,df_pay,how='left').fillna(0)
df

,shop_id,date,pay_count
0,1,2016-09-27,231.0
1,1,2016-09-28,164.0
2,1,2016-09-29,182.0
3,1,2016-09-30,303.0
4,1,2016-10-08,247.0
...,...,...,...
55995,2000,2016-10-27,132.0
55996,2000,2016-10-28,133.0
55997,2000,2016-10-29,67.0
55998,2000,2016-10-30,67.0


In [7]:
df_train = pd.pivot_table(df[df['date'].isin(train_date_range)],index='shop_id',columns='date',values='pay_count').reset_index()
columns = ['shop_id']
columns.extend([f'day_{_}' for _ in range(1,22)])
df_train.columns = columns
df_train.head()

,shop_id,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,...,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21
0,1,231.0,164.0,182.0,303.0,247.0,256.0,239.0,254.0,239.0,...,264.0,264.0,207.0,244.0,177.0,207.0,230.0,202.0,243.0,216.0
1,2,67.0,81.0,67.0,112.0,34.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,61.0,155.0,143.0,78.0
2,3,86.0,60.0,73.0,152.0,70.0,77.0,68.0,68.0,63.0,...,119.0,97.0,75.0,59.0,71.0,88.0,103.0,127.0,125.0,47.0
3,4,92.0,105.0,109.0,132.0,112.0,88.0,78.0,86.0,113.0,...,165.0,200.0,89.0,90.0,112.0,59.0,80.0,211.0,221.0,81.0
4,5,257.0,131.0,1.0,77.0,298.0,276.0,312.0,223.0,218.0,...,185.0,211.0,274.0,201.0,210.0,221.0,187.0,161.0,240.0,454.0


In [8]:
df_label = df[df['date'].isin(test_date_range)].groupby('shop_id')['pay_count'].sum().reset_index()
df_label

,shop_id,pay_count
0,1,1622.0
1,2,649.0
2,3,585.0
3,4,885.0
4,5,1251.0
...,...,...
1995,1996,685.0
1996,1997,1601.0
1997,1998,3762.0
1998,1999,2147.0


In [9]:
df_final = pd.merge(df_train,df_label,on='shop_id')
df_final.head()

,shop_id,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,...,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,pay_count
0,1,231.0,164.0,182.0,303.0,247.0,256.0,239.0,254.0,239.0,...,264.0,207.0,244.0,177.0,207.0,230.0,202.0,243.0,216.0,1622.0
1,2,67.0,81.0,67.0,112.0,34.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,61.0,155.0,143.0,78.0,649.0
2,3,86.0,60.0,73.0,152.0,70.0,77.0,68.0,68.0,63.0,...,97.0,75.0,59.0,71.0,88.0,103.0,127.0,125.0,47.0,585.0
3,4,92.0,105.0,109.0,132.0,112.0,88.0,78.0,86.0,113.0,...,200.0,89.0,90.0,112.0,59.0,80.0,211.0,221.0,81.0,885.0
4,5,257.0,131.0,1.0,77.0,298.0,276.0,312.0,223.0,218.0,...,211.0,274.0,201.0,210.0,221.0,187.0,161.0,240.0,454.0,1251.0


In [14]:
def construct_data(df_pay,train_start,train_end,test_start,test_end):
    '''
    构造数据集
    '''
    # 全量时间窗口
    all_date_range = [x.strftime('%Y-%m-%d') for x in pd.date_range(train_start,test_end)]
    # 忽略十一假期
    filter_date_range = [x.strftime('%Y-%m-%d') for x in pd.date_range('2016-10-01','2016-10-07')]
    all_date_range = list(filter(lambda x: x not in filter_date_range,all_date_range))

    # 训练集时间窗口
    train_date_range = all_date_range[:21]
    # 测试集时间窗口
    test_date_range = all_date_range[21:]
    
    # 构建店铺-日期维度
    df_date = pd.DataFrame(all_date_range,columns = ['date'])
    df_date['key']=0
    assert df_date.shape[0]==28
    df_shop = pd.DataFrame(range(1, 2001), columns=['shop_id'])
    df_shop['key'] = 0
    df_dim = pd.merge(df_shop, df_date, on='key')
    df_dim = df_dim[['shop_id', 'date']]
    assert df_dim.shape[0] == 2000*28
    
    # 日期维度与实际支付数据进行合并
    df = pd.merge(df_dim,df_pay,how='left').fillna(0)
    df_train = pd.pivot_table(df[df['date'].isin(train_date_range)],index='shop_id',columns='date',values='pay_count').reset_index()
#     columns = ['shop_id']
#     columns.extend([f'day_{_}' for _ in range(1,22)])
#     df_train.columns = columns
#     df_train.head()
    df_label = df[df['date'].isin(test_date_range)].groupby('shop_id')['pay_count'].sum().reset_index()
    df_final = pd.merge(df_train,df_label,on='shop_id')
    return df_final

In [15]:
df_pay = pd.read_csv('../data/pay_stat_day.csv')
'''
test_data X[2016-09-27,2019-09-30;2016-10-08,2016-10-24] Y[2016-10-25,2016-10-31]

train1 X[2016-09-20，2016-09-30；2016-10-08，2016-10-17] Y[2016-10-18,2016-10-24]

train2 X[2016-09-10,2016-09-30] Y[2016-10-08,2016-10-14]

train3 X[2016-09-03,2016-09-23] Y[2016-09-24,2016-09-30]
'''

data_config = {
    'test':{
        'train_start_date':'2016-09-27',
        'train_end_date':'2016-10-24',
        'test_start_date':'2016-10-25',
        'test_end_date':'2016-10-31'
    },
    'train1':{
        'train_start_date':'2016-09-20',
        'train_end_date':'2016-10-17',
        'test_start_date':'2016-10-18',
        'test_end_date':'2016-10-24'
    },
    'train2':{
        'train_start_date':'2016-09-10',
        'train_end_date':'2016-09-30',
        'test_start_date':'2016-10-08',
        'test_end_date':'2016-10-14'
    },
    'train3':{
        'train_start_date':'2016-09-03',
        'train_end_date':'2016-09-23',
        'test_start_date':'2016-09-24',
        'test_end_date':'2016-09-30'
    },
}
for k,v in data_config.items():
    train_start_date = v.get('train_start_date')
    train_end_date = v.get('train_end_date')
    test_start_date = v.get('test_start_date')
    test_end_date = v.get('test_end_date')
    tmp = construct_data(df_pay, train_start_date,
                         train_end_date, test_start_date, test_end_date)
    tmp.to_csv(f'../data/{k}_{train_start_date}_{test_end_date}.csv',index=False)